In [1]:
#import rerun as rr
from pathlib import Path
from argparse import ArgumentParser
import torch

from mini_dust3r.api import OptimizedResult, inferece_dust3r, log_optimized_result
from mini_dust3r.model import AsymmetricCroCo3DStereo


def process(image_dir: Path):
    device = "cpu"

    model = AsymmetricCroCo3DStereo.from_pretrained(
        "nielsr/DUSt3R_ViTLarge_BaseDecoder_512_dpt"
    ).to(device)

    optimized_results: OptimizedResult = inferece_dust3r(
        image_dir_or_list=image_dir,
        model=model,
        device=device,
        batch_size=1,
    )
    #log_optimized_result(optimized_results, Path("world"))
    return optimized_results

Warning, cannot find cuda-compiled version of RoPE2D, using a slow pytorch version instead


/home/diego/workspace/batimetria/venv/lib/python3.12/site-packages/mini_dust3r/cloud_opt/base_opt.py:299: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @torch.cuda.amp.autocast(enabled=False)


In [3]:
dir_path = Path('/home/diego/Downloads/gundam12/')
optimized_res = process(dir_path)

>> Loading images from /home/diego/Downloads/gundam12
 - adding IMG_20241008_153124.jpg with resolution 3000x4000 --> 384x512
 - adding IMG_20241008_153132.jpg with resolution 3000x4000 --> 384x512
 - adding IMG_20241008_153140.jpg with resolution 3000x4000 --> 384x512
 - adding IMG_20241008_153150.jpg with resolution 3000x4000 --> 384x512
 - adding IMG_20241008_153159.jpg with resolution 3000x4000 --> 384x512
 - adding IMG_20241008_153205.jpg with resolution 3000x4000 --> 384x512
 - adding IMG_20241008_153223.jpg with resolution 3000x4000 --> 384x512
 (Found 7 images)
>> Inference with model on 42 image pairs


  0%|          | 0/42 [00:00<?, ?it/s]/home/diego/workspace/batimetria/venv/lib/python3.12/site-packages/mini_dust3r/inference.py:70: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=bool(use_amp)):
/home/diego/workspace/batimetria/venv/lib/python3.12/site-packages/mini_dust3r/model.py:252: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
/home/diego/workspace/batimetria/venv/lib/python3.12/site-packages/mini_dust3r/inference.py:74: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=False):
  2%|▏         | 1/42 [00:12<08:16, 12.11s/it]/home/diego/workspace/batimetria/venv/lib/python3.12/site-packages/mini_dust3r/inference.py:70: FutureWarning: `t

 init edge (5*,4*) score=309.7616882324219
 init edge (0*,5) score=121.25961303710938
 init edge (5,6*) score=113.7037353515625
 init edge (2*,6) score=104.44024658203125
 init edge (6,3*) score=82.94054412841797
 init edge (0,1*) score=214.7468719482422
 init loss = 0.018322007730603218
Global alignement - optimizing for:
['pw_poses', 'im_depthmaps', 'im_poses', 'im_focals', 'im_conf.0', 'im_conf.1', 'im_conf.2', 'im_conf.3', 'im_conf.4', 'im_conf.5', 'im_conf.6']


100%|██████████| 7/7 [00:00<00:00, 12.49it/s]


In [4]:
# save point cloud
import pickle as pkl
with open('gundam_cloud.pkl', 'wb') as f:
    pkl.dump(optimized_res.point_cloud, f)

In [5]:
# open point cloud

with open('gundam_cloud.pkl', 'rb') as f:
    cloud = pkl.load(f)

In [7]:
#str = optimized_res.mesh.export("mesh.obj","obj")

In [15]:
import numpy as np
cloud = optimized_res.point_cloud
colors = cloud.colors
np_cloud = [np.array(c) for c in cloud]
np_cloud = np.array(np_cloud)

In [16]:
# Limit the number of points to visualize (e.g., 100 points)
max_points = 1000000  # Set your desired limit here
if len(np_cloud) > max_points:
    indices = np.random.choice(len(np_cloud), max_points, replace=False)
    new_np_cloud = np_cloud[indices]
    new_colors = colors[indices][:, 1]
colors = np.array(new_colors)

In [17]:
import numpy as np
import pyvista

# create a point cloud
point_cloud = new_np_cloud
pdata = pyvista.PolyData(point_cloud)
pdata['point_color'] = colors
#pc = pdata.glyph(scale=False,orient=False)
#pc.plot(cmap='Reds')
pdata.plot(show_bounds=True, cpos='xy', point_size=2, cmap='gray')

Widget(value='<iframe src="http://localhost:36953/index.html?ui=P_0x7b19b373df40_3&reconnect=auto" class="pyvi…